## Load Environments

In [1]:
from __future__ import print_function
from __future__ import division
import warnings; warnings.filterwarnings('ignore')

import time
import random
from math import log

import numpy as np
import pandas as pd

random.seed(125)
np.random.seed(137)

In [15]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV, LeaveOneGroupOut, ParameterGrid
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_approximation import RBFSampler

In [16]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from xgboost import XGBClassifier

In [17]:
from numerapi.numerapi import NumerAPI
from sklearn.externals import joblib

In [18]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout


## Read in raw data sets, training, tournament (val, test, live)

In [7]:
!ls /input

numerai_tournament_data.csv  numerai_training_data.csv


In [8]:
def import_data_sets():
    train = pd.read_csv('/input/numerai_training_data.csv', index_col=0).drop('data_type', axis=1)
    df = pd.read_csv('/input/numerai_tournament_data.csv', index_col=0)
    valid = df.loc[df['data_type']=='validation'].drop('data_type', axis=1)
    test = df.loc[df['data_type']=='test'].drop('data_type', axis=1)
    live = df.loc[df['data_type']=='live'].drop('data_type', axis=1)
    return(train, valid, test, live)

In [9]:
train, valid, test, live = import_data_sets()

In [10]:
feature_cols = [f for f in train.columns if "feature" in f]

In [11]:
x_train = train[feature_cols]
x_val = valid[feature_cols]
x_test = test[feature_cols]
x_live = live[feature_cols]
y_train = train['target']
y_val = valid['target']


train_eras = train['era'].values
val_eras = valid['era'].values

In [12]:
def calc_consistency(labels, preds, eras):
    """ Calculate the consistency score.

    Args:
        labels: (np array) The correct class ids
        preds:  (np array) The predicted probabilities for class 1
        eras:   (np array) The era each sample belongs to
    """
    unique_eras = np.unique(eras)
    better_than_random_era_count = 0
    for era in unique_eras:
        this_era_filter = [eras == era]
        logloss = log_loss(labels[this_era_filter], preds[this_era_filter])
        
        print(logloss)
        
        if logloss < -np.log(0.5):
            better_than_random_era_count += 1

    consistency = better_than_random_era_count / float(len(unique_eras)) * 100
    print(consistency)

## Kernal Approximation with SGD Classifier

In [ ]:
# sgd_model = joblib.load('sgd_model.pkl') 

In [32]:
pipe = Pipeline([
    ('scaler', MinMaxScaler()),
    ('rbf', RBFSampler(random_state=73)),
    ('classify', SGDClassifier(loss='log', penalty='l2'))
])

N_COMPONENTS = [6, 7, 8]
GAMMA = [0.1, 0.09, 0.08]
ALPHA = [0.0009, 0.001, 0.002]

param_grid = [{
        'rbf__n_components': N_COMPONENTS,
        'rbf__gamma': GAMMA,
        'classify__alpha': ALPHA
    }]

sgd_grid = GridSearchCV(pipe, cv=5, n_jobs=1,
                       param_grid=param_grid, verbose=2, scoring='neg_log_loss')

In [33]:
sgd_grid.fit(x_train.values, y_train.values, groups=train_eras)
sgd_model = sgd_grid.best_estimator_

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] classify__alpha=0.0009, rbf__gamma=0.1, rbf__n_components=6 .....
[CV]  classify__alpha=0.0009, rbf__gamma=0.1, rbf__n_components=6, total=   0.8s
[CV] classify__alpha=0.0009, rbf__gamma=0.1, rbf__n_components=6 .....


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s


[CV]  classify__alpha=0.0009, rbf__gamma=0.1, rbf__n_components=6, total=   0.8s
[CV] classify__alpha=0.0009, rbf__gamma=0.1, rbf__n_components=6 .....
[CV]  classify__alpha=0.0009, rbf__gamma=0.1, rbf__n_components=6, total=   0.8s
[CV] classify__alpha=0.0009, rbf__gamma=0.1, rbf__n_components=6 .....
[CV]  classify__alpha=0.0009, rbf__gamma=0.1, rbf__n_components=6, total=   0.8s
[CV] classify__alpha=0.0009, rbf__gamma=0.1, rbf__n_components=6 .....
[CV]  classify__alpha=0.0009, rbf__gamma=0.1, rbf__n_components=6, total=   0.8s
[CV] classify__alpha=0.0009, rbf__gamma=0.1, rbf__n_components=7 .....
[CV]  classify__alpha=0.0009, rbf__gamma=0.1, rbf__n_components=7, total=   0.9s
[CV] classify__alpha=0.0009, rbf__gamma=0.1, rbf__n_components=7 .....
[CV]  classify__alpha=0.0009, rbf__gamma=0.1, rbf__n_components=7, total=   0.9s
[CV] classify__alpha=0.0009, rbf__gamma=0.1, rbf__n_components=7 .....
[CV]  classify__alpha=0.0009, rbf__gamma=0.1, rbf__n_components=7, total=   0.9s
[CV] cl

[CV]  classify__alpha=0.001, rbf__gamma=0.1, rbf__n_components=8, total=   0.9s
[CV] classify__alpha=0.001, rbf__gamma=0.1, rbf__n_components=8 ......
[CV]  classify__alpha=0.001, rbf__gamma=0.1, rbf__n_components=8, total=   0.9s
[CV] classify__alpha=0.001, rbf__gamma=0.1, rbf__n_components=8 ......
[CV]  classify__alpha=0.001, rbf__gamma=0.1, rbf__n_components=8, total=   0.9s
[CV] classify__alpha=0.001, rbf__gamma=0.1, rbf__n_components=8 ......
[CV]  classify__alpha=0.001, rbf__gamma=0.1, rbf__n_components=8, total=   0.9s
[CV] classify__alpha=0.001, rbf__gamma=0.1, rbf__n_components=8 ......
[CV]  classify__alpha=0.001, rbf__gamma=0.1, rbf__n_components=8, total=   0.9s
[CV] classify__alpha=0.001, rbf__gamma=0.09, rbf__n_components=6 .....
[CV]  classify__alpha=0.001, rbf__gamma=0.09, rbf__n_components=6, total=   0.8s
[CV] classify__alpha=0.001, rbf__gamma=0.09, rbf__n_components=6 .....
[CV]  classify__alpha=0.001, rbf__gamma=0.09, rbf__n_components=6, total=   0.8s
[CV] classif

[CV]  classify__alpha=0.002, rbf__gamma=0.09, rbf__n_components=7, total=   0.9s
[CV] classify__alpha=0.002, rbf__gamma=0.09, rbf__n_components=7 .....
[CV]  classify__alpha=0.002, rbf__gamma=0.09, rbf__n_components=7, total=   0.9s
[CV] classify__alpha=0.002, rbf__gamma=0.09, rbf__n_components=7 .....
[CV]  classify__alpha=0.002, rbf__gamma=0.09, rbf__n_components=7, total=   0.9s
[CV] classify__alpha=0.002, rbf__gamma=0.09, rbf__n_components=7 .....
[CV]  classify__alpha=0.002, rbf__gamma=0.09, rbf__n_components=7, total=   0.9s
[CV] classify__alpha=0.002, rbf__gamma=0.09, rbf__n_components=7 .....
[CV]  classify__alpha=0.002, rbf__gamma=0.09, rbf__n_components=7, total=   0.9s
[CV] classify__alpha=0.002, rbf__gamma=0.09, rbf__n_components=8 .....
[CV]  classify__alpha=0.002, rbf__gamma=0.09, rbf__n_components=8, total=   0.9s
[CV] classify__alpha=0.002, rbf__gamma=0.09, rbf__n_components=8 .....
[CV]  classify__alpha=0.002, rbf__gamma=0.09, rbf__n_components=8, total=   0.9s
[CV] cl

[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:  2.7min finished


In [29]:
print(sgd_grid.best_params_)
print(sgd_grid.best_score_)

{'classify__alpha': 0.001, 'rbf__gamma': 0.1, 'rbf__n_components': 7}
-0.692983572493


In [30]:
calc_consistency(y_val.values, sgd_model.predict_proba(x_val.values)[:,1], val_eras)

0.692922844509
0.692289576552
0.693214244666
0.693105158595
0.693122744147
0.69308969077
0.694218019443
0.693126919855
0.692655049302
0.693553707768
0.692618037985
0.692958772219
75.0


In [31]:
joblib.dump(sgd_model, 'sgd_model.pkl')

['sgd_model.pkl']

## Keras NN Model

In [34]:
nn_model = load_model('nn_model.h5') 

In [15]:
def create_nn(input_size):
    
    model = Sequential()
    model.add(Dense(1048, activation='relu', input_dim=input_size))
    model.add(Dense(1048, activation='relu', input_dim=input_size))
    model.add(Dense(64, activation='relu', input_dim=input_size))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop', loss='binary_crossentropy')
    
    return(model)
    

In [17]:
nn_model = create_nn(x_train.values.shape[1])

nn_model.fit(x_train.values, y_train.values, epochs=50, batch_size=256,
             verbose=2, validation_data=(x_val.values, y_val.values))

Train on 393613 samples, validate on 46362 samples
Epoch 1/50
8s - loss: 0.6937 - val_loss: 0.6930
Epoch 2/50
8s - loss: 0.6928 - val_loss: 0.6927
Epoch 3/50
8s - loss: 0.6927 - val_loss: 0.6926
Epoch 4/50
8s - loss: 0.6926 - val_loss: 0.6926
Epoch 5/50
8s - loss: 0.6926 - val_loss: 0.6926
Epoch 6/50
8s - loss: 0.6926 - val_loss: 0.6925
Epoch 7/50
8s - loss: 0.6926 - val_loss: 0.6925
Epoch 8/50
8s - loss: 0.6925 - val_loss: 0.6926
Epoch 9/50
8s - loss: 0.6925 - val_loss: 0.6928
Epoch 10/50
8s - loss: 0.6925 - val_loss: 0.6929
Epoch 11/50
8s - loss: 0.6925 - val_loss: 0.6926
Epoch 12/50
8s - loss: 0.6925 - val_loss: 0.6930
Epoch 13/50
8s - loss: 0.6925 - val_loss: 0.6925
Epoch 14/50
8s - loss: 0.6925 - val_loss: 0.6926
Epoch 15/50


KeyboardInterrupt: 

In [19]:
nn_model.save('nn_model.h5')

In [19]:
calc_consistency(y_val.values, nn_model.predict(x_val.values)[:,0], val_eras)

0.693187226658
0.69185995405
0.692754740139
0.691465966345
0.692912073941
0.691386033033
0.69422584474
0.692626403434
0.690243344033
0.692883365658
0.692701965481
0.694108005542
75.0


## GridSearch CV Pipeline - XGBoost Classifier

In [35]:
xgb_model = joblib.load('xgb_model.pkl') 

In [20]:
pipe = Pipeline([
    ('scaler', MinMaxScaler()),
    ('classify', XGBClassifier(eval_metric='logloss', n_jobs=-1, silent=False))
])

N_EST = [60]
LR = [0.01]
DEPTH = [10]

param_grid = [{
        'classify__n_estimators': N_EST,
        'classify__learning_rate': LR,
        'classify__max_depth': DEPTH        
    }]

xgb_grid = GridSearchCV(pipe, cv=5, n_jobs=1,
                        param_grid=param_grid, verbose=2, scoring='neg_log_loss')

In [21]:
xgb_grid.fit(x_train.values, y_train.values, groups=train_eras)

xgb_model = xgb_grid.best_estimator_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] classify__learning_rate=0.01, classify__max_depth=10, classify__n_estimators=60 
[CV]  classify__learning_rate=0.01, classify__max_depth=10, classify__n_estimators=60, total=  52.8s
[CV] classify__learning_rate=0.01, classify__max_depth=10, classify__n_estimators=60 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   53.5s remaining:    0.0s


[CV]  classify__learning_rate=0.01, classify__max_depth=10, classify__n_estimators=60, total=  54.1s
[CV] classify__learning_rate=0.01, classify__max_depth=10, classify__n_estimators=60 
[CV]  classify__learning_rate=0.01, classify__max_depth=10, classify__n_estimators=60, total=  54.3s
[CV] classify__learning_rate=0.01, classify__max_depth=10, classify__n_estimators=60 
[CV]  classify__learning_rate=0.01, classify__max_depth=10, classify__n_estimators=60, total=  54.3s
[CV] classify__learning_rate=0.01, classify__max_depth=10, classify__n_estimators=60 
[CV]  classify__learning_rate=0.01, classify__max_depth=10, classify__n_estimators=60, total=  54.7s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  4.6min finished


In [22]:
print(xgb_grid.best_params_)
print(xgb_grid.best_score_)

{'classify__learning_rate': 0.01, 'classify__max_depth': 10, 'classify__n_estimators': 60}
-0.692990104896


In [23]:
joblib.dump(xgb_model, 'xgb_model.pkl')

['xgb_model.pkl']

In [24]:
calc_consistency(y_val.values, xgb_model.predict_proba(x_val.values)[:,1], val_eras)

0.692757578673
0.692333242709
0.692659891456
0.692046344997
0.692832007653
0.692471211325
0.693559660013
0.693367822003
0.692296648975
0.692565317921
0.693395092986
0.693128827844
75.0


## Random Forest Model

In [36]:
rf_model = joblib.load('rf_model.pkl') 

In [63]:
pipe = Pipeline([
    ('scaler', MinMaxScaler()),
    ('reduce_dim', PCA()),
    ('classify', RandomForestClassifier(n_jobs=-1, verbose=2, random_state=73))
])

N_DIM = [10]
N_EST = [50]
MAX_DEPTH = [5]

param_grid = [{
        'reduce_dim__n_components': N_DIM,
        'classify__n_estimators': N_EST,
        'classify__max_depth': MAX_DEPTH
    }]

rf_grid = GridSearchCV(pipe, cv=5, n_jobs=1,
                        param_grid=param_grid, verbose=2, scoring='neg_log_loss')

In [64]:
rf_grid.fit(x_train.values, y_train.values, groups=train_eras)

rf_model = rf_grid.best_estimator_

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] classify__n_estimators=25, classify__max_depth=5, reduce_dim__n_components=10 
building tree 1 of 25building tree 2 of 25building tree 3 of 25building tree 4 of 25building tree 5 of 25building tree 7 of 25building tree 8 of 25






building tree 6 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25building tree 14 of 25

building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done  23 out of  25 | elapsed:    2.6s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    3.0s finished
[Parallel(n_jobs=8)]: Done  23 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Done  23 out of  25 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.3s remaining:    0.0s


[CV]  classify__n_estimators=25, classify__max_depth=5, reduce_dim__n_components=10, total=   5.9s
[CV] classify__n_estimators=25, classify__max_depth=5, reduce_dim__n_components=10 
building tree 1 of 25building tree 2 of 25building tree 3 of 25building tree 4 of 25building tree 5 of 25building tree 7 of 25building tree 6 of 25






building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25
building tree 15 of 25
building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


[Parallel(n_jobs=-1)]: Done  23 out of  25 | elapsed:    2.5s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    3.0s finished
[Parallel(n_jobs=8)]: Done  23 out of  25 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Done  23 out of  25 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  25 out of  25 | elapsed:    0.1s finished


[CV]  classify__n_estimators=25, classify__max_depth=5, reduce_dim__n_components=10, total=   5.5s
[CV] classify__n_estimators=25, classify__max_depth=5, reduce_dim__n_components=10 
building tree 1 of 25building tree 2 of 25building tree 3 of 25building tree 4 of 25building tree 5 of 25building tree 6 of 25building tree 7 of 25






building tree 8 of 25
building tree 9 of 25
building tree 10 of 25
building tree 11 of 25
building tree 12 of 25
building tree 13 of 25
building tree 14 of 25building tree 15 of 25

building tree 16 of 25
building tree 17 of 25
building tree 18 of 25
building tree 19 of 25
building tree 20 of 25
building tree 21 of 25
building tree 22 of 25
building tree 23 of 25
building tree 24 of 25
building tree 25 of 25


...

[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed: 39.6min finished


building tree 1 of 50building tree 2 of 50building tree 3 of 50building tree 4 of 50building tree 5 of 50building tree 8 of 50





building tree 6 of 50
building tree 7 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.2s


building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50building tree 47 of 50

building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    7.1s finished


In [65]:
print(rf_grid.best_params_)
print(rf_grid.best_score_)

{'classify__n_estimators': 50, 'classify__max_depth': 5, 'reduce_dim__n_components': 10}
-0.692823565237


In [66]:
joblib.dump(rf_model, 'rf_model.pkl')

['rf_model.pkl']

In [26]:
calc_consistency(y_val.values, rf_model.predict_proba(x_val.values)[:,1], val_eras)

0.69217419868
0.69136241285
0.692977870444
0.692662864181
0.69208463949
0.692729669791
0.693354743035
0.692913053723
0.691437053556
0.692642917427
0.692786441803
0.693989096434
83.33333333333334


[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished


## Ensemble

In [38]:
to_pred = pd.concat([x_val, x_test, x_live])

#to_pred = x_val

xgb_preds = pd.Series(xgb_model.predict_proba(to_pred.values)[:,1])
sgd_preds = pd.Series(sgd_model.predict_proba(to_pred.values)[:,1])
rf_preds = pd.Series(rf_model.predict_proba(to_pred.values)[:,1])
nn_preds = pd.Series(nn_model.predict(to_pred.values)[:,0])

to_vote = pd.concat([nn_preds, sgd_preds, rf_preds, xgb_preds], axis=1)
to_vote.columns = ['nn', 'sgd', 'rf', 'xgb']

# Ensemble Tree
ens_mean = to_vote.mean(axis=1)

[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.2s finished


In [94]:
y_val.index.shape

(46362,)

In [91]:
to_vote.shape

(46362, 4)

In [95]:
compare_votes = pd.concat([to_vote, ens_mean, ens_pred], axis=1)

compare_votes = compare_votes.set_index(y_val.index)

compare_votes = pd.concat([compare_votes, y_val], axis=1)

compare_votes.columns = ['nn', 'lr', 'rf', 'xgb', 'mean_vote', 'extreme_vote', 'y']

ValueError: Length mismatch: Expected axis has 243470 elements, new values have 46362 elements

In [86]:
y_val.head(n=10)

id
n61d97b708905409    1.0
n350462f2ea6f467    0.0
n9451f9eda57c499    0.0
n3eeda8f0c131475    0.0
nad2c6fb811fb4bb    0.0
n32ae86b8e7fa48c    1.0
n773959f97ab4466    1.0
n92288bc86be34f4    1.0
n969ab572b92344c    1.0
n16d4a532357a4a4    1.0
Name: target, dtype: float64

In [87]:
compare_votes.head(n=10)

,nn,lr,rf,xgb,mean_vote,extreme_vote
0,0.494663,0.494995,0.498833,0.497314,0.496451,0.494663
1,0.483115,0.494382,0.502927,0.467982,0.487101,0.467982
2,0.470381,0.476135,0.494698,0.489114,0.482582,0.470381
3,0.516463,0.514595,0.514231,0.507132,0.513105,0.516463
4,0.484614,0.487337,0.479529,0.501046,0.488131,0.479529
5,0.528005,0.525912,0.515544,0.517901,0.521841,0.528005
6,0.512459,0.511508,0.513146,0.509831,0.511736,0.513146
7,0.515951,0.515522,0.508164,0.504132,0.510942,0.515951
8,0.494394,0.498773,0.506513,0.502808,0.500622,0.506513
9,0.517273,0.510546,0.502577,0.517385,0.511945,0.517385


In [47]:
def apply_vote(row):
    
    mean = np.mean(row)
    
    if(mean > .5):
        return np.max(row)
    else:
        return np.min(row)

ens_pred = to_vote.apply(func=apply_vote,axis=1)

In [51]:
nn_preds.describe()

count    243470.000000
mean          0.497833
std           0.018012
min           0.422405
25%           0.486345
50%           0.499384
75%           0.509356
max           0.548866
dtype: float64

In [52]:
lr_preds.describe()

count    243470.000000
mean          0.499958
std           0.014833
min           0.439059
25%           0.489796
50%           0.500190
75%           0.510283
max           0.558966
dtype: float64

In [54]:
xgb_preds.describe()

count    243470.000000
mean          0.500057
std           0.012328
min           0.414145
25%           0.493722
50%           0.501080
75%           0.507569
max           0.605291
dtype: float64

In [46]:
ens_mean.describe()

count    243470.000000
mean          0.501599
std           0.011523
min           0.449006
25%           0.494332
50%           0.502583
75%           0.509677
max           0.539226
dtype: float64

In [47]:
ens_preds.describe()

count    243470.000000
mean          0.501599
std           0.011523
min           0.449006
25%           0.494332
50%           0.502583
75%           0.509677
max           0.539226
dtype: float64

In [39]:
ens_preds = ens_mean

In [40]:
sub = pd.concat([pd.Series(to_pred.index), pd.Series(ens_preds)], axis=1)
sub.columns = ['id', 'probability']
sub.head()

,id,probability
0,n61d97b708905409,0.497261
1,n350462f2ea6f467,0.489940
2,n9451f9eda57c499,0.486143
3,n3eeda8f0c131475,0.513111
4,nad2c6fb811fb4bb,0.487891


In [41]:
pred_path = 'sub_sk_88_' + str(int(time.time())) + '.csv'

sub.to_csv(pred_path, index=False)

In [42]:
pred_path

'sub_sk_88_1514743311.csv'

In [43]:
def get_model_key(model):
    
    if(model == "spacekitty"):
        return('6GHLWWOGPWHBN5ODRPJFCXVV6U4C5WBB','HOZSYDP45H7FCVV3N2QWRZVXTRYWS74QODA2ON5TOUHRJLG3STJOU63M2ZVYTZGN')


In [44]:
def upload_preds(path_to_preds, pub, priv):
    # set example username and round
    example_public_id = pub
    example_secret_key = priv

    # some API calls do not require logging in
    napi = NumerAPI(verbosity="info")
    
    # download current dataset
    napi.download_current_dataset(unzip=True)
    
    # provide api tokens
    napi = NumerAPI(example_public_id, example_secret_key)

    # upload predictions
    submission_id = napi.upload_predictions(path_to_preds)

In [45]:
pub, priv = get_model_key('spacekitty')

upload_preds(pred_path, pub, priv)

2017-12-31 18:02:02,820 INFO numerapi.numerapi: downloading current dataset...
2017-12-31 18:02:02,826 INFO requests.packages.urllib3.connectionpool: Starting new HTTPS connection (1): api-tournament.numer.ai
2017-12-31 18:02:03,107 INFO requests.packages.urllib3.connectionpool: Starting new HTTPS connection (1): numerai-datasets.s3.amazonaws.com
2017-12-31 18:02:05,814 INFO numerapi.numerapi: unzipping file...
2017-12-31 18:02:08,600 INFO numerapi.numerapi: uploading prediction...
2017-12-31 18:02:08,602 INFO requests.packages.urllib3.connectionpool: Starting new HTTPS connection (1): api-tournament.numer.ai
2017-12-31 18:02:08,627 INFO requests.packages.urllib3.connectionpool: Starting new HTTPS connection (1): numerai-production-uploads.s3.amazonaws.com
2017-12-31 18:02:09,367 INFO requests.packages.urllib3.connectionpool: Starting new HTTPS connection (1): api-tournament.numer.ai
